In [10]:
%pip install accelerate

%pip install evaluate


Note: you may need to restart the kernel to use updated packages.


In [11]:
# Importing required packages

import transformers
from datasets import load_dataset
import torch
from accelerate import Accelerator
import evaluate

In [12]:
train_dataset = load_dataset('csv', data_files='train.csv')
test_dataset = load_dataset('csv', data_files='test.csv')

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [13]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score
import time

# Initialize tokenizers and models

distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base')

# Function to evaluate model performance

def evaluate_model(model, tokenizer, texts, labels):
    start_time = time.time()
    encoded_batch = tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors='pt')
    input_ids = encoded_batch['input_ids']
    attention_mask = encoded_batch['attention_mask']
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = logits.argmax(dim=-1)
    
    accuracy = accuracy_score(labels, predictions)
    inference_time = time.time() - start_time
    
    return accuracy, inference_time

# Example dataset 

test_texts = ["The quick brown fox jumps over the lazy dog.", "To be or not to be, that is the question."]
test_labels = [0, 1] 

# Evaluate DistilBERT

distilbert_accuracy, distilbert_inference_time = evaluate_model(distilbert_model, distilbert_tokenizer, test_texts, test_labels)
print(f"DistilBERT Accuracy: {distilbert_accuracy}")
print(f"DistilBERT Inference Time: {distilbert_inference_time} seconds\n")

# Evaluate BERT

bert_accuracy, bert_inference_time = evaluate_model(bert_model, bert_tokenizer, test_texts, test_labels)
print(f"BERT Accuracy: {bert_accuracy}")
print(f"BERT Inference Time: {bert_inference_time} seconds\n")

# Evaluate RoBERTa

roberta_accuracy, roberta_inference_time = evaluate_model(roberta_model, roberta_tokenizer, test_texts, test_labels)
print(f"RoBERTa Accuracy: {roberta_accuracy}")
print(f"RoBERTa Inference Time: {roberta_inference_time} seconds\n")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

DistilBERT Accuracy: 0.5
DistilBERT Inference Time: 0.038001298904418945 seconds

BERT Accuracy: 0.5
BERT Inference Time: 0.05199933052062988 seconds

RoBERTa Accuracy: 0.5
RoBERTa Inference Time: 0.05500006675720215 seconds



Based on the metrics, ill use Distilbert as the balance between size speed and accuracy make it a good choice for the project.